In [ ]:
from transformers import AutoImageProcessor,  AutoTokenizer, AutoModelForCausalLM, ViTModel,  BertConfig, ViTConfig, VisionEncoderDecoderConfig, VisionEncoderDecoderModel, VisionEncoderDecoderModel ,ViTImageProcessor

from transformers import AutoImageProcessor,  AutoTokenizer, AutoModelForCausalLM, ViTModel,  BertConfig, ViTConfig, VisionEncoderDecoderConfig, VisionEncoderDecoderModel, VisionEncoderDecoderModel ,ViTImageProcessor


from typing import * 
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import io, transforms
import torchvision.datasets as dset
from tqdm.auto import tqdm
from transformers import EncoderDecoderModel, GPT2Tokenizer, ViTFeatureExtractor



import torch







import torch






In [ ]:
from transformers import AutoImageProcessor,  AutoTokenizer, AutoModelForCausalLM, ViTModel,  BertConfig, ViTConfig, VisionEncoderDecoderConfig, VisionEncoderDecoderModel, VisionEncoderDecoderModel ,ViTImageProcessor




In [ ]:
configs = {
  "_name_or_path": "NinedayWang/PolyCoder-0.4B",
  "add_cross_attention": True,
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "bos_token_id": 0,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "rotary_emb_base": 10000,
  "rotary_pct": 1.0,
  "tie_word_embeddings": False,
  "tokenizer_class": "GPT2Tokenizer",
    "is_cross_attention":True,
  "torch_dtype": "float32",
  "transformers_version": "4.27.3",
  "use_cache": True,
  "use_parallel_residual": False,
  "vocab_size": 50304
}

In [ ]:
from DesignCoder.model import  GPTNeoXCrossAttentionForCausalLM
from DesignCoder.model_with_new_attention import GPTNeoXEmbedAttentionForCausalLM
from codelms.DesignCoder.model import GPTNeoXForCausalLM


In [ ]:
GPTNeoXForCausalLM

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

from packaging import version
assert version.parse(transformers.__version__) >= version.parse("4.23.0")


"""model = GPTNeoXForCausalLM.from_pretrained("Code-LMs/Convert2HF/polycoder/0-4B/")
tokenizer = AutoTokenizer.from_pretrained("NinedayWang/PolyCoder-2.7B")
"""

model_path = "/Users/mehmetburaksayici/Desktop/sahibinden/ccrraawwll/_designcoder/"
tokenizer_path = "/Users/mehmetburaksayici/Desktop/sahibinden/ccrraawwll/_designcoder_tokenizer/"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

decoder = GPTNeoXEmbedAttentionForCausalLM.from_pretrained(model_path) # GPTNeoXCrossAttentionForCausalLM


In [ ]:
decoder



prompt = '''def binarySearch(arr, left, right, x):
    mid = (left +'''
input_ids = tokenizer.encode(prompt, return_tensors='pt')
result = decoder.generate(input_ids, max_length=50, num_beams=1, num_return_sequences=1)
for res in result:
    print(tokenizer.decode(res))


In [ ]:
107520/192

In [ ]:
105*16*192

In [ ]:

#tokenizer = AutoTokenizer.from_pretrained("NinedayWang/PolyCoder-0.4B")
image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

tokenizer = AutoTokenizer.from_pretrained("NinedayWang/PolyCoder-0.4B")


encoder = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

encoder_config = encoder.config
encoder_config.add_cross_attention = True
encoder = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k",config = encoder_config)




# Initializing a model with a pretrained Swin as encoder & a pretrained BART-large as decoder
model = VisionEncoderDecoderModel(encoder=encoder, decoder=decoder)






In [ ]:

MEAN = (0.485, 0.456, 0.406)
STD = (0.229, 0.224, 0.225)

TRAIN_PCT = 0.95
NUM_WORKERS = 2
BATCH_SIZE = 8
EPOCHS = 3
LR = 1e-4
IMAGE_SIZE = (224, 224)

MAX_TEXT_LENGTH = 32

LABEL_MASK = -100

TOP_K = 1000
TOP_P = 0.95


tfms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE), 
        transforms.ToTensor(),
        transforms.Normalize(
            mean=0.5, 
            std=0.5
        )
   ]
)
descale = transforms.Compose(
    [
        transforms.Normalize(
            mean = [ 0., 0., 0. ],
            std = 1 / 0.5
        ),
        transforms.Normalize(
            mean = -0.5,
            std = [ 1., 1., 1. ]
        ),                           
    ]
)

In [ ]:


images =  Image.fromarray(np.zeros((224,224,3)).astype(np.uint8))

pixel_values = image_processor(images).pixel_values
inp_ = torch.from_numpy(pixel_values[0])[None,]
"""encoder_outputs = encoder(inp_)
encoder_outputs.keys()"""

In [ ]:
inp_ = torch.from_numpy(pixel_values[0])[None,]
model.generate(inp_)



In [ ]:
pixel_values[]

In [ ]:
decoder.gpt_neox.layers[0]

In [ ]:
torch.mm(torch.zeros(1,197,1024),torch.zeros(1,1,1024),2)

In [ ]:
(torch.zeros(1,197,1024)*torch.zeros(1,1,1024)).shape

In [ ]:

"""

def generate_sentence_from_image(model, encoder_outputs, tokenizer, max_text_length: int, device)-> List[str]:
    generated_so_far = torch.LongTensor([[tokenizer.bos_token_id]]*len(encoder_outputs.last_hidden_state)).to(device)
    with torch.no_grad():
        for _ in tqdm(range(max_text_length)):
            attention_mask = torch.ones_like(generated_so_far)
            decoder_out = model(
                decoder_input_ids=generated_so_far, 
                decoder_attention_mask=attention_mask,
                encoder_outputs=encoder_outputs
            )

            next_token_logits = decoder_out["logits"][:, -1, :]
            filtered_p = top_k_top_p_filtering(next_token_logits, top_k=TOP_K, top_p=TOP_P, device=device)
            next_token = torch.multinomial(filtered_p, num_samples=1)
            generated_so_far = torch.cat((generated_so_far, next_token), dim=1)

    return [tokenizer.decode(coded_sentence) for coded_sentence in generated_so_far]




images =  Image.fromarray(np.zeros((224,224,3)).astype(np.uint8))

pixel_values = image_processor(images).pixel_values

encoder_outputs = model.encoder(pixel_values=tfms(images)[None,:])



generated_sentences = generate_sentence_from_image(
                  model, 
                encoder_outputs, 
                tokenizer, 
                MAX_TEXT_LENGTH,
                "cpu"
            )"""